# Azure Machine Learning Pipeline with AutoMLStep (AutoML training in pipeline)
This notebook demonstrates the use of **AutoMLStep** for training in Azure Machine Learning Pipeline.
As secondary pipeline step, it also uses a **PythonScriptStep** for registering the trained model into AML Workspace model registry.

## Introduction
In this example we showcase AML Pipelines, how you can train a model with AutoML as one pipeline step with AutoMLStep while using AML Datasets as input data. 

If you are using an Azure Machine Learning Compute Instance (aka. Notebook VM), you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) and [Azure Automated ML setup](https://github.com/Azure/MachineLearningNotebooks/tree/master/how-to-use-azureml/automated-machine-learning#setup-using-a-local-conda-environment) before running this notebook.

In this notebook you will learn how to:

1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a **TabularDataset**.
4. Configure AutoML using **AutoMLConfig**.
5. Configure **AutoMLStep** step for training
6. Configure **PythonScriptStep** for registering the model in the Workspace
7. Run the AML pipeline using AmlCompute
8. Explore the results.
9. Test the best fitted model.
10. Publish the Pipeline in the Workspace

## Azure Machine Learning and Pipeline SDK-specific imports

In [ ]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.dataset import Dataset
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

from azureml.pipeline.steps import AutoMLStep

## Check Azure ML SDK version

In [ ]:
import azureml.core
print("This notebook was created and tested using version 1.2.0 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create an Azure ML experiment
Let's create an experiment named "automlstep-classif-porto".

In [ ]:
# Choose a name for the run history container in the workspace.
experiment_name = 'automlstep-classif-porto'
experiment = Experiment(ws, experiment_name)
experiment

### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
# Define remote compute target to use
# Further docs on Remote Compute Target: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-remote

# Choose a name for your cluster.
amlcompute_cluster_name = "cpu-cluster"

found = False
# Check if this compute target already exists in the workspace.
cts = ws.compute_targets

if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
     found = True
     print('Found existing training cluster.')
     # Get existing cluster
     # Method 1:
     aml_remote_compute = cts[amlcompute_cluster_name]
     # Method 2:
     # aml_remote_compute = ComputeTarget(ws, amlcompute_cluster_name)
    
if not found:
     print('Creating a new training cluster...')
     provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D13_V2", # for GPU, use "STANDARD_NC12"
                                                                 #vm_priority = 'lowpriority', # optional
                                                                 max_nodes = 5)
     # Create the cluster.
     aml_remote_compute = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)
    
print('Checking cluster status...')
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
aml_remote_compute.wait_for_completion(show_output = True, min_node_count = 0, timeout_in_minutes = 20)

# For a more detailed view of current AmlCompute status, use get_status().

## Data

### (***Optional***) Submit dataset file into DataStore (Azure Blob under the covers)

In [ ]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir='../../data/', 
                 target_path='Datasets/porto_seguro_safe_driver_prediction', overwrite=True, show_progress=True)

## Load data into Azure ML Dataset and Register into Workspace

In [ ]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file in the HTTP URL
found = False
aml_dataset_name = "porto_seguro_safe_driver_prediction_train"

if aml_dataset_name in ws.datasets.keys(): 
       found = True
       dataset = ws.datasets[aml_dataset_name] 
       print("Dataset found and loaded from the Workspace")
       
if not found:
        # Create AML Dataset and register it into Workspace
        print("Dataset does not exist in the current Workspace. It will be imported and registered.")
        
        # Option A: Create AML Dataset from file in AML DataStore
        datastore = ws.get_default_datastore()
        dataset = Dataset.Tabular.from_delimited_files(path=datastore.path('Datasets/porto_seguro_safe_driver_prediction/porto_seguro_safe_driver_prediction_train.csv'))
        data_origin_type = 'AMLDataStore'
        
        # Option B: Create AML Dataset from file in HTTP URL
        # data_url = 'https://azmlworkshopdata.blob.core.windows.net/safedriverdata/porto_seguro_safe_driver_prediction_train.csv'
        # aml_dataset = Dataset.Tabular.from_delimited_files(data_url)  
        # data_origin_type = 'HttpUrl'
        
        print(aml_dataset)
                
        #Register Dataset in Workspace
        registration_method = 'SDK'  # or 'UI'
        dataset = aml_dataset.register(workspace=ws,
                                           name=aml_dataset_name,
                                           description='Porto Seguro Safe Driver Prediction Train dataset file',
                                           tags={'Registration-Method': registration_method, 'Data-Origin-Type': data_origin_type},
                                           create_new_version=True)
        
        print("Dataset created from file and registered in the Workspace")

In [ ]:
dataset.take(5).to_pandas_dataframe().head(3)

### Segregate a Test dataset for later testing and creating a confusion matrix
Split original AML Tabular Dataset in two test/train AML Tabular Datasets (using AML DS function)

In [ ]:
# The name and target column of the Dataset to create 
train_dataset_name = "porto_seguro_safe_driver_prediction_train90"

In [ ]:
# Split using Azure Tabular Datasets (Better for Remote Compute)
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.tabulardataset?view=azure-ml-py#random-split-percentage--seed-none-

train_dataset, test_dataset = dataset.random_split(0.9, seed=1)

#Register Train Dataset (90%) after Split in Workspace
registration_method = 'SDK'  # or 'UI'
data_origin_type = 'SPLIT'
train_dataset = train_dataset.register(workspace=ws,
                                       name=train_dataset_name,
                                       description='Porto Seguro Safe Driver Prediction Train dataset file (90%)',
                                       tags={'Registration-Method': registration_method, 'Data-Origin-Type': data_origin_type},
                                       create_new_version=True)

# Load from Workspace
train_dataset = ws.datasets[train_dataset_name] 
train_dataset

In [ ]:
# Use Pandas DF only to check the data
train_df = train_dataset.to_pandas_dataframe()
test_df = test_dataset.to_pandas_dataframe()

print(train_df.shape)
print(test_df.shape)

train_df.describe()

### List possible metrics to optimize for (primary metric) in Classification using AutoML

In [ ]:
from azureml.train import automl

# List of possible primary metrics is here:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#primary-metric
    
# Get a list of valid metrics for your given task
automl.utilities.get_primary_metrics('classification')

## Train configuration in AutoMLConfig
This creates a general AutoML settings object.

Note that we're using a single algorithm (LightGBM as the only whitelisted algo) and a single iteration (iterations=1) just for the sake of time when trying the notebook so the end-to-end run needs less time.

In a real training run you would let AutoML to try many iterations in order to find the "best model".

In [ ]:
import logging
from azureml.train.automl import AutoMLConfig

# Path to the Azure ML project folder
project_folder = './project'

# You can provide additional settings as a **kwargs parameter for the AutoMLConfig object
automl_settings = {
      "blacklist_models":['LogisticRegression', 'ExtremeRandomTrees', 'RandomForest'], 
      # "whitelist_models": ['LightGBM'],
      # "n_cross_validations": 5,
      # "validation_data": test_dataset,   # Better to holdout the Test Dataset
      "experiment_exit_score": 0.7
}

automl_config = AutoMLConfig(compute_target=aml_remote_compute,
                             task='classification',
                             primary_metric='AUC_weighted',                           
                             training_data=train_dataset, # AML Dataset
                             label_column_name="target",
                             path = project_folder,
                             enable_early_stopping= True,
                             iterations=20,
                             max_concurrent_iterations=5,
                             experiment_timeout_hours=1,                           
                             featurization= 'auto',   # (auto/off) All feature columns in this dataset are numbers, no need to featurize with AML Dataset. 
                             debug_log='automated_ml_errors.log',
                             verbosity= logging.INFO,
                             model_explainability=True,
                             enable_onnx_compatible_models=False,
                             **automl_settings
                             )

# Explanation of Settings: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train#configure-your-experiment-settings

# AutoMLConfig info on: 
# https://docs.microsoft.com/en-us/python/api/azureml-train-automl-client/azureml.train.automl.automlconfig.automlconfig

# The AML Pipeline

### PipelineData objects

Let's first define the data (metrics and model) to be produced by the pipeline.

The **PipelineData** object is a special kind of data reference that is used for interim storage locations that can be passed between pipeline steps.

Note that we also need to pass it as a script argument so our code can access the datastore location referenced by the data reference.

In [ ]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                            datastore=ds,
                            pipeline_output_name=metrics_output_name,
                            training_output=TrainingOutput(type='Metrics'))

model_data = PipelineData(name='model_data',
                          datastore=ds,
                          pipeline_output_name=best_model_output_name,
                          training_output=TrainingOutput(type='Model'))

print(model_data.get_env_variable_name())

## Create an AutoMLStep for training.
Pipelines consist of one or more *steps*, which can be specialized steps like an AutoMLStep (like this step) for training a model, or Python scripts, or a data transfer step that copies data from one location to another. 

Each step can run in its own compute context.

In [ ]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=False)

## Create a PythonScriptStep to register the model in the Workspace.

Create Environment and Conda dependencies needed to run the PythonScriptStep.

### Environment, Conda-Dependencies and RunConfiguration for PythonScriptStep

In [ ]:
from azureml.core.runconfig import CondaDependencies, DEFAULT_CPU_IMAGE, RunConfiguration
from azureml.core import Environment

# Create an Environment for future usage
custom_env = Environment("python-script-step-env")
custom_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies
custom_env.docker.enabled = True 
custom_env.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE 

conda_dependencies = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', 'applicationinsights'], #'azureml-explain-model'
                                              conda_packages=['numpy==1.16.2'], 
                                              pin_sdk_version=False)

# Add the dependencies to the environment
custom_env.python.conda_dependencies = conda_dependencies

# Register the environment (To use it again)
custom_env.register(workspace=ws)
registered_env = Environment.get(ws, 'python-script-step-env')

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = aml_remote_compute

# Assign the environment to the run configuration
conda_run_config.environment
conda_run_config.environment = registered_env

print('Run config is ready')

## Register Model Step (Python Script Step)
Script to register the model to the workspace.

First, let's create a folder where the script will be placed.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

In [ ]:
import os
import shutil
scripts_folder="Scripts"
os.makedirs(scripts_folder, exist_ok=True)

### Create the register_model.py script file

In [ ]:
%%writefile $scripts_folder/register_model.py
from azureml.core.model import Model, Dataset
from azureml.core.run import Run, _OfflineRun
from azureml.core import Workspace
from azureml.core.resource_configuration import ResourceConfiguration
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--model_name")
parser.add_argument("--model_path")
parser.add_argument("--ds_name")
args = parser.parse_args()

print("Argument 1(model_name): %s" % args.model_name)
print("Argument 2(model_path): %s" % args.model_path)
print("Argument 3(ds_name): %s" % args.ds_name)

run = Run.get_context()
ws = None
if type(run) == _OfflineRun:
    ws = Workspace.from_config()
else:
    ws = run.experiment.workspace

train_ds = Dataset.get_by_name(ws, args.ds_name)
datasets = [(Dataset.Scenario.TRAINING, train_ds)]

# Register model with training dataset

model = Model.register(workspace=ws,
                       model_path=args.model_path,                  # File to upload and register as a model.
                       model_name=args.model_name,                  # Name of the registered model in your workspace.
                       datasets=datasets,
                       description='Porto Seguro Safe Driving Prediction.',
                       tags={'area': 'insurance', 'type': 'classification'}
                      )

print("Registered version {0} of model {1}".format(model.version, model.name))

### PythonScriptStep to run register_model.py script

In [ ]:
from azureml.pipeline.core import PipelineParameter

# The model name with which to register the trained model in the workspace.
model_name = "porto-model-from-automlstep"
model_name_param = PipelineParameter("model_name", default_value=model_name)

# The Dataset name to relate with the model to register in the workspace.
dataset_name_param = PipelineParameter(name="ds_name", default_value=train_dataset_name)

In [ ]:
# Create PythonScriptStep for Model registration

from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps import PythonScriptStep

register_model_step = PythonScriptStep(name="register_model",                                      
                                       source_directory = scripts_folder,            # Local folder with .py script
                                       script_name="register_model.py",
                                       allow_reuse=False,
                                       arguments=["--model_name", model_name_param, "--model_path", model_data, "--ds_name", dataset_name_param],
                                       inputs=[model_data],
                                       compute_target=aml_remote_compute,
                                       runconfig=conda_run_config)

register_model_step.run_after(automl_step)

print("Pipeline steps defined")

This is a simple example, designed to demonstrate the principle. In reality, you could build more sophisticated logic into the pipeline steps - for example, evaluating the model against some test data to calculate a performance metric like AUC or accuracy, comparing the metric to that of any previously registered versions of the model, and only registering the new model if it performs better.

## Create Pipeline and add the multiple steps into it

In [ ]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step, register_model_step]) 

print("Pipeline is built.")

In [ ]:
pipeline_run = experiment.submit(pipeline, pipeline_parameters={
        "ds_name": train_dataset_name, "model_name": model_name})

print("Pipeline submitted for execution.")

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion()

## Examine Results from Pipeline

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [ ]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
import json
with open(metrics_output._path_on_datastore) as f:  
   metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

### Retrieve info about the trained model

In [ ]:
print(pipeline_run.get_file_names())

In [ ]:
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
best_model_output
# num_file_downloaded = best_model_output.download('.', show_progress=True)

### Retrieve the Best Model

In [ ]:
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

In [ ]:
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

### Test the Model (Make predictions, get probabilities and calculate metrics)

#### Prep Test Data: Extract X values (feature columns) from test dataset and convert to NumPi array for predicting 

In [ ]:
import pandas as pd

x_test_df = test_df.copy()

if 'target' in x_test_df.columns:
    y_test_df = x_test_df.pop('target')

print(test_df.shape)
print(x_test_df.shape)
print(y_test_df.shape)

# Testing Our Best Fitted Model

In [ ]:
# Try the best model making predictions with the test dataset
y_predictions = best_model.predict(x_test_df)

print('10 predictions: ')
print(y_predictions[:10])

### Calculate Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

print('Accuracy with Scikit-Learn model:')
print(accuracy_score(y_test_df, y_predictions))


### Calculate AUC with Test Dataset

In [ ]:
# Get Probabilities/scores per class

class_probabilities = best_model.predict_proba(x_test_df)
print(class_probabilities.shape)

print('Some class probabilities...: ')
print(class_probabilities[:3])

In [ ]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score

print('ROC AUC:')
print(roc_auc_score(y_test_df, class_probabilities[:,1]))

print('ROC AUC Weighted:')
print(roc_auc_score(y_test_df, class_probabilities[:,1], average='weighted'))

## Show Confusion Matrix
We will use confusion matrix to see how our model works.

In [ ]:
# Need to have installed: 
!pip install pandas_ml

In [ ]:
from pandas_ml import ConfusionMatrix

cm = ConfusionMatrix(y_test_df, y_predictions)

print(cm)

cm.plot()

## Publish the Pipeline
Now that you've created a pipeline and verified it works, you can publish it as a REST service.

In [ ]:
published_pipeline = pipeline.publish(name="Training_Pipeline_AutoMLStep_Porto",
                                      description="Pipeline with AutoMLStep for Training model for Porto Seguro Safe Driver")
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

# Trigger the AML Pipeline by using the Pipeline REST Endpoint

To use the endpoint, client applications need to make a REST call over HTTP. This request must be authenticated, so an authorization header is required. A real application would require a service principal with which to be authenticated, but to test this out, we'll use the authorization header from your current connection to your Azure workspace, which you can get using the following code:

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print(auth_header)

Now you're ready to call the REST interface. The pipeline runs asynchronously, so you'll get an identifier back, which you can use to track the pipeline experiment as it runs:

### The REST Endpoint
Note that the published pipeline has an endpoint, which you can see in the Endpoints page (on the Pipeline Endpoints tab) in Azure Machine Learning studio. You can also find its URI as a property of the published pipeline object.
So, you could also copy that REST Endpoint from the AML portal and paste it like here:

rest_endpoint = "Your opied REST Endpoint here"
    

In [ ]:
import requests

response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": experiment_name})
run_id = response.json()["Id"]
run_id

Since you have the run ID, you can use the RunDetails widget to view the experiment as it runs.

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments[experiment_name], run_id)
RunDetails(published_pipeline_run).show()

# Next Steps!
You can use the Azure Machine Learning extension for Azure DevOps to combine Azure ML pipelines with Azure DevOps pipelines (yes, it is confusing that they have the same name!) and integrate model retraining into a continuous integration/continuous deployment (CI/CD) process. For example you could use an Azure DevOps build pipeline to trigger an Azure ML pipeline that trains and registers a model, and when the model is registered it could trigger an Azure Devops release pipeline that deploys the model as a web service, along with the application or service that consumes the model.